In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('kinetic_data.xlsx')

In [3]:
# properties
vt = 145 # mL, total reactor volume
rho_h2o = 1 # g/mL, water density
mw_h2o = 18 # g/mol, water molweight
rho_gly = 1.26 # g/mL, glycerol density
mw_gly = 92 # g/mol, glycerol molweight
mw_koh = 55 # g/mol, KOH molweight
mw_h2 = 2 # g/mol, hydrogen molweight

# h2 solubility parameter
p00 = 0.2638
p01 = 5.443e-4
p02 = -2.45e-7
p10 = -1.545e-3
p20 = 2.205e-6
p11 = 9.105e-7

In [4]:
# treatment
df['W_H2O [g]'] = df['VL_H2O [mL]']
df['Ni_H2O [mmol]'] = df['W_H2O [g]']/mw_h2o*1000
df['EG [%]'] = df['ME [%]']
df['TOT [%]'] = df['LA [%]'] + df['PDO [%]'] + (df['FA [%]'] + df['GA [%]'])/2 + df['ME [%]']
df['Wi [g]'] = df['W_H2O [g]'] + df['Ni_Gly [mmol]']*mw_gly/1000 + df['Ni_KOH [mmol]']*mw_koh/1000
df['VL [mL]'] = df['VL_H2O [mL]'] +df['Ni_Gly [mmol]']*mw_gly/1000/rho_gly
df['VG [mL]'] = vt - df['VL [mL]']

# selectivity
df_sel = pd.DataFrame()
df_sel['LA [%]'] = df['LA [%]']
df_sel['PDO [%]'] = df['PDO [%]'] + (100-df['TOT [%]'])*df['PDO [%]']/(df['PDO [%]'] + (df['FA [%]'] + df['GA [%]'])/2 + df['ME [%]'])
df_sel['FA [%]'] = df['FA [%]'] + (100-df['TOT [%]'])*(df['FA [%]']+df['GA [%]'])/2/(df['PDO [%]'] + (df['FA [%]'] + df['GA [%]'])/2 + df['ME [%]'])
df_sel['GA [%]'] = df['GA [%]'] + (100-df['TOT [%]'])*(df['FA [%]']+df['GA [%]'])/2/(df['PDO [%]'] + (df['FA [%]'] + df['GA [%]'])/2 + df['ME [%]'])
df_sel['ME [%]'] = df['ME [%]'] + (100-df['TOT [%]'])*df['ME [%]']/(df['PDO [%]'] + (df['FA [%]'] + df['GA [%]'])/2 + df['ME [%]'])
df_sel['EG [%]'] = df_sel['ME [%]']
df_sel.iloc[0] = 0
df_sel['TOT [%]'] = df_sel['LA [%]'] + df_sel['PDO [%]'] + (df_sel['FA [%]'] + df_sel['GA [%]'])/2 + df_sel['ME [%]']

# total mole number
df_mol = pd.DataFrame()
df_mol['N_Gly [mmol]'] = df['Ni_Gly [mmol]']*(100-df['Conv [%]'])/100
df_mol['N_LA [mmol]'] = df['Ni_Gly [mmol]']*df['Conv [%]']/100*df_sel['LA [%]']/100
df_mol['N_PDO [mmol]'] = df['Ni_Gly [mmol]']*df['Conv [%]']/100*df_sel['PDO [%]']/100
df_mol['N_FA [mmol]'] = df['Ni_Gly [mmol]']*df['Conv [%]']/100*df_sel['FA [%]']/100
df_mol['N_GA [mmol]'] = df['Ni_Gly [mmol]']*df['Conv [%]']/100*df_sel['GA [%]']/100
df_mol['N_ME [mmol]'] = df['Ni_Gly [mmol]']*df['Conv [%]']/100*df_sel['ME [%]']/100
df_mol['N_EG [mmol]'] = df['Ni_Gly [mmol]']*df['Conv [%]']/100*df_sel['EG [%]']/100
df_mol['N_KOH [mmol]'] = df['Ni_KOH [mmol]'] - df_mol['N_LA [mmol]'] - df_mol['N_FA [mmol]'] - df_mol['N_GA [mmol]']
df_mol['N_H2O [mmol]'] = df['Ni_H2O [mmol]'] + df_mol['N_LA [mmol]'] + df_mol['N_PDO [mmol]']
df_mol['N_H2 [mmol]'] = df_mol['N_LA [mmol]'] + df_mol['N_FA [mmol]']*1.5 + df_mol['N_GA [mmol]']*1.5 - df_mol['N_ME [mmol]'] - df_mol['N_PDO [mmol]']
df_mol['N_TOT [mmol]'] = df_mol.sum(axis=1)

# hydrogen solubility

from thermo import Chemical
from thermo.eos import PR

h2 = Chemical('hydrogen')
h2o = Chemical('water')

df_h2 = pd.DataFrame()
df_h2['P_H2O [bar]'] = 10**(8.14019-1810.94/(244.485+df['Temp [degC]']))/750.062 # Antoine equation
df_h2['NG_H2O [mmol]'] = 0.0
df_h2['P_H2 [bar]'] = 0.0
for i in range(len(df_h2)):
    vg = PR(T=(df['Temp [degC]'].iloc[i]+273.15),P=df_h2['P_H2O [bar]'].iloc[i]*1e5,Tc=h2o.Tc,Pc=h2o.Pc,omega=h2o.omega).V_g # molar volume from PR EOS
    df_h2.loc[i,'NG_H2O [mmol]'] = 1/(vg*1e3)*df.loc[i,'VG [mL]']
    if i != 0:
        vm = df.loc[i,'VG [mL]']/1e6/df_mol.loc[i,'N_H2 [mmol]']*1e3
        df_h2.loc[i, 'P_H2 [bar]'] = PR(T=(df.loc[i,'Temp [degC]']+273.15),V=vm,Tc=h2.Tc,Pc=h2.Pc,omega=h2.omega).P/1e5
    else:
        pass
df_h2['S_H2 [mol/kg]'] = p00 + (df['Temp [degC]']+273.15)*p10 + df_h2['P_H2 [bar]']*p01 + (df['Temp [degC]']+273.15)**2*p20 + df_h2['P_H2 [bar]']**2*p02 + (df['Temp [degC]']+273.15)*df_h2['P_H2 [bar]']*p11
df_h2['WL [g]'] = df['Wi [g]'] - df_mol['N_H2 [mmol]']*mw_h2/1000 - df_h2['NG_H2O [mmol]']*mw_h2o/1000
df_h2['NL_H2 [mmol]'] = df_h2['S_H2 [mol/kg]']*df_h2['WL [g]']
df_h2.loc[0,'NL_H2 [mmol]'] = 0
df_h2

,P_H2O [bar],NG_H2O [mmol],P_H2 [bar],S_H2 [mol/kg],WL [g],NL_H2 [mmol]
0,15.518495,44.143356,0.000000,0.026419,65.830420,0.000000
1,15.518495,44.143356,44.507140,0.069332,65.598556,4.548103
2,15.518495,44.143356,67.668346,0.091280,65.480244,5.977062
3,15.518495,44.143356,72.865992,0.096170,65.453920,6.294682
4,15.518495,44.143356,78.882032,0.101812,65.423555,6.660920
5,15.518495,44.143356,78.565588,0.101516,65.425149,6.641693
6,7.869734,23.166000,31.641614,0.041900,66.031414,2.766693
7,9.974426,28.975289,47.475245,0.061345,65.845490,4.039313
8,12.504831,35.912444,62.017012,0.080189,65.650237,5.264442
9,15.518495,44.143356,78.882032,0.101812,65.423555,6.660920


In [5]:
# Dataset
data = pd.DataFrame()
data['label'] = ['time']*6 + ['temp']*4 + ['cat']*4
data['T [degC]'] = df['Temp [degC]']
data['P [bar]'] = df_h2['P_H2 [bar]'] + df_h2['P_H2O [bar]']
data['W_cat [g]'] = df['W_cat [g]']
data['Time [h]'] = df['Time [hr]']
data['C_Gly [M]'] = df_mol['N_Gly [mmol]']/df['VL [mL]']
data['C_KOH [M]'] = df_mol['N_KOH [mmol]']/df['VL [mL]']
data['C_H2 [M]'] = df_h2['NL_H2 [mmol]']/df['VL [mL]']
data['C_LA [M]'] = df_mol['N_LA [mmol]']/df['VL [mL]']
data['C_PDO [M]'] = df_mol['N_PDO [mmol]']/df['VL [mL]']
data['C_FA [M]'] = df_mol['N_FA [mmol]']/df['VL [mL]']
data['C_GA [M]'] = df_mol['N_GA [mmol]']/df['VL [mL]']
data['C_ME [M]'] = df_mol['N_ME [mmol]']/df['VL [mL]']
data['C_EG [M]'] = df_mol['N_EG [mmol]']/df['VL [mL]']

In [6]:
data.to_csv('data.csv',index=False)